In [ ]:
import numpy as np

# -----------------------------
# Utilities
# -----------------------------
def softmax(x: np.ndarray) -> np.ndarray:
    x = x - np.max(x)
    ex = np.exp(x)
    return ex / np.sum(ex)

def one_hot(i: int, V: int) -> np.ndarray:
    x = np.zeros((V, 1), dtype=np.float64)
    x[i, 0] = 1.0
    return x

# -----------------------------
# Vanilla RNN (tanh) from scratch
# -----------------------------
class VanillaRNN:
    def __init__(self, vocab_size: int, hidden_size: int = 64, seed: int = 0):
        rng = np.random.default_rng(seed)
        V, H = vocab_size, hidden_size

        # Parameters
        # x_t is one-hot (Vx1), h_t is (Hx1)
        self.Wxh = rng.normal(0, 0.01, size=(H, V))   # input -> hidden
        self.Whh = rng.normal(0, 0.01, size=(H, H))   # hidden -> hidden
        self.Why = rng.normal(0, 0.01, size=(V, H))   # hidden -> output (logits)
        self.bh  = np.zeros((H, 1))
        self.by  = np.zeros((V, 1))

        # For Adagrad/SGD convenience (optional)
        self._grads = None

    def forward(self, inputs, hprev):
        """
        inputs: list[int] of length T (token indices)
        hprev: (H,1)
        Returns: loss, cache
        cache contains all intermediates for BPTT.
        """
        V = self.by.shape[0]
        H = self.bh.shape[0]
        T = len(inputs)

        xs, hs, ys, ps = {}, {}, {}, {}
        hs[-1] = hprev

        for t in range(T):
            xs[t] = one_hot(inputs[t], V)                       # (V,1)
            hs[t] = np.tanh(self.Wxh @ xs[t] + self.Whh @ hs[t-1] + self.bh)  # (H,1)
            ys[t] = self.Why @ hs[t] + self.by                 # (V,1) logits
            ps[t] = softmax(ys[t])                              # (V,1)

        cache = (xs, hs, ys, ps)
        return cache

    def loss(self, ps, targets):
        """
        Cross-entropy over sequence.
        targets: list[int] length T
        """
        T = len(targets)
        L = 0.0
        for t in range(T):
            p = ps[t][targets[t], 0]
            L += -np.log(p + 1e-12)
        return L / T

    def bptt(self, cache, targets):
        """
        Backprop through time to compute gradients.
        """
        xs, hs, ys, ps = cache
        V = self.by.shape[0]
        H = self.bh.shape[0]
        T = len(targets)

        dWxh = np.zeros_like(self.Wxh)
        dWhh = np.zeros_like(self.Whh)
        dWhy = np.zeros_like(self.Why)
        dbh  = np.zeros_like(self.bh)
        dby  = np.zeros_like(self.by)

        dh_next = np.zeros((H, 1))

        for t in reversed(range(T)):
            # output gradient: dy = p - y_true
            dy = ps[t].copy()
            dy[targets[t], 0] -= 1.0                            # (V,1)

            dWhy += dy @ hs[t].T                                # (V,H)
            dby  += dy                                          # (V,1)

            dh = self.Why.T @ dy + dh_next                      # (H,1)
            dh_raw = (1 - hs[t] * hs[t]) * dh                   # tanh' = 1 - h^2

            dbh  += dh_raw
            dWxh += dh_raw @ xs[t].T                            # (H,V)
            dWhh += dh_raw @ hs[t-1].T                          # (H,H)

            dh_next = self.Whh.T @ dh_raw

        # Clip to avoid exploding gradients
        for dparam in (dWxh, dWhh, dWhy, dbh, dby):
            np.clip(dparam, -5, 5, out=dparam)

        self._grads = (dWxh, dWhh, dWhy, dbh, dby)
        return self._grads

    def step(self, lr=1e-1):
        dWxh, dWhh, dWhy, dbh, dby = self._grads
        self.Wxh -= lr * dWxh
        self.Whh -= lr * dWhh
        self.Why -= lr * dWhy
        self.bh  -= lr * dbh
        self.by  -= lr * dby

    def sample(self, start_idx, n, h=None):
        """
        Generate n characters starting from start_idx.
        """
        V = self.by.shape[0]
        H = self.bh.shape[0]
        if h is None:
            h = np.zeros((H, 1))

        x = one_hot(start_idx, V)
        out = [start_idx]
        for _ in range(n - 1):
            h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
            y = self.Why @ h + self.by
            p = softmax(y).ravel()
            idx = int(np.random.choice(V, p=p))
            out.append(idx)
            x = one_hot(idx, V)
        return out

# -----------------------------
# Tiny demo: character LM
# -----------------------------
if __name__ == "__main__":
    data = "hello Georgetown from scratch!\n"
    chars = sorted(list(set(data)))
    stoi = {ch: i for i, ch in enumerate(chars)}
    itos = {i: ch for ch, i in stoi.items()}

    encoded = [stoi[ch] for ch in data]
    V = len(chars)

    rnn = VanillaRNN(vocab_size=V, hidden_size=32, seed=42)

    seq_len = 12
    lr = 0.2
    h = np.zeros((32, 1))

    for epoch in range(2000):
        # simple single-sequence training (wrap around)
        start = epoch % (len(encoded) - seq_len - 1)
        inp = encoded[start : start + seq_len]
        tgt = encoded[start + 1 : start + seq_len + 1]

        cache = rnn.forward(inp, h)
        xs, hs, ys, ps = cache
        L = rnn.loss(ps, tgt)

        rnn.bptt(cache, tgt)
        rnn.step(lr=lr)

        # carry hidden state (detach by copying; here it's numpy so it's fine)
        h = hs[seq_len - 1]

        if epoch % 50 == 0:
            sample_ids = rnn.sample(start_idx=inp[0], n=40, h=h.copy())
            sample_txt = "".join(itos[i] for i in sample_ids)
            print(f"epoch {epoch:3d} loss {L:.3f} | sample: {sample_txt!r}")